## Miniproject 1

#### Part 1: Transport of London

Figure out on your own how to analyze and look through the API data.  
Use the questions in the Mini-Presentation as an inspiration - focus on a larger use case and scenario  

5 minute presentation - make a story
- Present as if to client or teammate (assume base knowledge)
- Recommends have no code (just slides): going line by line is boring
- Motivation, Task, Modeling, Results, Conclusions
    - draw.io is a good one for quick schematics n models
    - python plotting for data visualization and results (week 3)

##### *Tips*
Make functions!
- If you can give a short name to block of code
- If you're writing the same code repeated

Save GET requests and only fetch when needed!
- lowercase json module will turn into a string can save into file and object

Keep projects into Github for traffic

##

## Should I take the bus to work?

Inputs:
- Bus duration x
- AirQuality x
- Disruptions
    - bus x
    - road
    
### Set up

In [1]:
#import necessary packages
import requests as re
from IPython.display import JSON

#import api key
from api_keys import transport_primary, transport_secondary
print(transport_primary, transport_secondary) #make sure it's working

# endpoint for the transport API
root_endpoint = "https://api.tfl.gov.uk/"

09e7db574b42473e91fc6d50c49fb8aa 0d4fa2721cb04275a8ee4c6b89a571ba


### AirQuality

In [2]:
#Let's test it using AirQuality
search_term = 'AirQuality'

response = re.get(root_endpoint + search_term, params={'app_key':transport_primary})

#Let's make sure it worked
response.status_code

200

In [3]:
#Ok now let's see what we're working with
results = response.json()

JSON(results)

<IPython.core.display.JSON object>

In [4]:
#Print the AirQuality predictions for tomorrow
future_results = results['currentForecast'][1]

print(f"Tomorrow the air quality will be {future_results['forecastBand']}")

Tomorrow the air quality will be Low


In [5]:
#This is a function that will intake string of API info we're looking for
#and a dictionary of the parameters to define
def TFL_API(search_term, parameters = {'app_key':transport_primary}):
    
    endpoint = root_endpoint + search_term
    
    response = re.get(root_endpoint + search_term, 
    params=parameters)
    
    status_code, results = response.status_code, response.json()
    
    #Let's make sure it worked
    if status_code != 200:
        print('Something went wrong!')
        print(status_code)
        
    return results

### Bus Duration

In [6]:
#Let's get the bus duration input
journey_test = TFL_API('Journey/JourneyResults/WC2N_4BF/to/W1C_1DH')

In [7]:
#taking a look at our data so we can get plan below
#JSON(journey_test)

In [64]:
#Let's make it a function so we can enter whatever addresses we would like and
#get back the fastest bus route assuming we want to minimize walking

def journey_status(start, end, mode = 'tube'):
    
    #define empty objects
    duration = []
    route_disruptions = []
    departing_station = []
    
    #define url
    journey_url = 'Journey/JourneyResults/' + start + '/to/' + end
    journey = TFL_API(journey_url, parameters={'journeyPreference': 'leasttime',
                                               'mode': mode,
                                               'app_key': transport_primary})
    
    
    #create and store durations
    for options in journey['journeys']:
        duration.append(options['duration'])
    
    #save the shortest duration
    fastest_route = min(duration)
    
    #now let's get the status of that route
    for options in journey['journeys']:
        
        #only the fastest route
        if options['duration'] == fastest_route:
            
            #only the status for the parts of the route that involve a bus
            for legs in options['legs']:
                
                #if it is a bus mode, add the disruption status
                if legs['mode']['id'] == mode:
                    route_disruptions.append(legs['isDisrupted'])
                    
                    try:
                        #Find and save the Naptan station id
                        station = legs['departurePoint']['naptanId']
                    except:
                        station = "No station id available"
                    
    
    #sum disruptions of all legs and save as boolean
    route_disrupted =  bool(sum(route_disruptions))
    
    return station, fastest_route, route_disrupted


In [65]:
#Test out the function. It works!
station, fastest_bus, route_disrupted = journey_status('WC2N_4BF', 'HA1 4DZ', 'cycle')

In [66]:
#Our fastest bus journey in terms of minutes
print(fastest_bus)
print(route_disrupted)
print(station)
JSON(journey_test)

92
False
No station id available


<IPython.core.display.JSON object>

### Crowding

In [11]:
def crowding_results(station):
    
    endpoint = 'crowding/' + station + '/Live'
    crowding_temp = TFL_API(endpoint)
    
    return (crowding_temp['percentageOfBaseline'] * 100)

In [28]:
crowding_percent = crowding_results('490G00128B')

In [29]:
print(crowding_percent)

0


### Bringing it together

In [71]:
def recommend_transport(start, stop, mode = 'tube'):
    
    #create base return string
    car_recommend = "Take a car, the"
    
    #load up the inputs
    air_quality = TFL_API('AirQuality')['currentForecast'][0]['forecastBand']
    station, fastest_route, route_disrupted = journey_status(start, stop, mode)    
    
    try:
        #based on the station of fastest_route
        depart_station_crowd = crowding_results(station)
    except:
        print("Live crowding data not viable")
        depart_station_crowd = 0
    
    print(f"The fastest route taking the {mode} is {fastest_route} minutes")
    
    if depart_station_crowd == 0:
        print("No live crowding data available")
        
    #If everything is within parameters, recommend transport
    if (air_quality != 'Low') and (fastest_route < 100) and (route_disrupted == False) and (depart_station_crowd <= 10.0):
        return f"Take the {mode}."
    
    #series of elif to return why to take the car, keep in mind order dictates priority
    elif fastest_route > 100:
        return f"{car_recommend}, {mode} will take too long"
    
    elif depart_station_crowd >= 10.0:
        return f"{car_recommend} departing station is too crowded."
    
    elif route_disrupted == True:
        return f"{car_recommend} {mode} is disrupted"
    
    
    elif air_quality == 'Low':
        return f"{car_recommend} air quality is bad."
    

In [73]:
recommend_transport('WC2N_4BF', 'W1C_1DH', 'walking')

The fastest route taking the walking is 23 minutes
No live crowding data available


'Take the walking.'

In [ ]:
#valid mode inputs
mode = ['bus', 'tube', 'cycle']

### Tests

In [ ]:
#test_results = TFL_API('crowding/940GZZLUCHX/Live')

In [ ]:
#JSON(test_results)